The alignment from phonetisaurus is quite good, but there are some things that don't quite fit with an English speaker's intuition, so we repair some of them.

In [5]:
bigrams_to_unigrams = """
e}_ l}el e|l}el
e}_ d}ed e|d}ed
e}iy1 e}_ e|e}iy1
i}ix o|n}n i|o}ix n}n
"""

In [8]:
def make_pair_replacements(fixables):
  out = {}
  for line in fixables.split('\n'):
    if line == '':
      continue
    tmp = line.split(' ')
    out[tmp[0]] = (tmp[1], tmp[2])
  return out

In [17]:
bg_to_ug = make_pair_replacements(bigrams_to_unigrams)

In [18]:
bg_to_ug

{'e}_': ('d}ed', 'e|d}ed'),
 'e}iy1': ('e}_', 'e|e}iy1'),
 'i}ix': ('o|n}n', 'i|o}ix')}

In [48]:
def fix_graphones(graphones):
  out = []
  skip = False
  for i in range(len(graphones)):
    if skip:
      skip = False
      continue
    if graphones[i] in bg_to_ug:
      possible_replacement = bg_to_ug[graphones[i]]
      if i < len(graphones) and graphones[i+1] == possible_replacement[0]:
        out.append(possible_replacement[1])
        skip = True
    elif graphones[i] == '-|c}ao':
      out.append('-}_')
      out.append('c}k')
    elif graphones[i] == 'c}ao':
      if graphones[0] == 'n}n':
        out.append('c}s')
      else:
        out.append('c}k')
    elif graphones[i].startswith('-|'):
      out.append('-}_')
      out.append(graphones[i][2:])
    else:
      out.append(graphones[i])
  return out

In [51]:
#fix_graphones("w}w e}iy1 e}_ w}w".split(' '))
fix_graphones("-|u}ah1 p}p".split(' '))


['-}_', 'u}ah1', 'p}p']

In [3]:
with open('TIMIT.clean.corpus', 'r') as f:
  for line in f.readlines():
     graphones = line.split(' ')
     for i in range(0, len(graphones)):
       print (f'{graphones}: {graphones[i]}')

Streaming output truncated to the last 5000 lines.
['t}t', 'e|r}axr', 'r}_', 'i}ih1', 'f}f', 'i}ih', 'c}k\n']: i}ih
['t}t', 'e|r}axr', 'r}_', 'i}ih1', 'f}f', 'i}ih', 'c}k\n']: c}k

['t}t', 'e}eh1', 'r|r}r', 'i}ax', 'f}f', 'y}ay2', 'i}ix', 'n|g}ng\n']: t}t
['t}t', 'e}eh1', 'r|r}r', 'i}ax', 'f}f', 'y}ay2', 'i}ix', 'n|g}ng\n']: e}eh1
['t}t', 'e}eh1', 'r|r}r', 'i}ax', 'f}f', 'y}ay2', 'i}ix', 'n|g}ng\n']: r|r}r
['t}t', 'e}eh1', 'r|r}r', 'i}ax', 'f}f', 'y}ay2', 'i}ix', 'n|g}ng\n']: i}ax
['t}t', 'e}eh1', 'r|r}r', 'i}ax', 'f}f', 'y}ay2', 'i}ix', 'n|g}ng\n']: f}f
['t}t', 'e}eh1', 'r|r}r', 'i}ax', 'f}f', 'y}ay2', 'i}ix', 'n|g}ng\n']: y}ay2
['t}t', 'e}eh1', 'r|r}r', 'i}ax', 'f}f', 'y}ay2', 'i}ix', 'n|g}ng\n']: i}ix
['t}t', 'e}eh1', 'r|r}r', 'i}ax', 'f}f', 'y}ay2', 'i}ix', 'n|g}ng\n']: n|g}ng

['t}t', 'e}eh1', 's}s', 't}t\n']: t}t
['t}t', 'e}eh1', 's}s', 't}t\n']: e}eh1
['t}t', 'e}eh1', 's}s', 't}t\n']: s}s
['t}t', 'e}eh1', 's}s', 't}t\n']: t}t

['t}t', 'e}eh1', 's}s', 't}t', '-|r}r', 'u}ah1', 'n}